In [125]:
import os
import pandas as pd
import numpy as np
from natsort import natsorted
from sklearn import metrics

In [126]:
data_path = "/Users/imrankabir/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data"
gt_fol = os.path.join(data_path, "GT")

In [235]:
coco_common_obj = ['Person', 'Bicycle', 'Car', 'Motorcycle', 'Bus', 'Traffic Signals', 'Fire hydrant', 'Stop sign'
                   'Bench', 'Dog', 'Chair', 'Vegetation']

pfb_common_obj = ['Road', 'Sidewalk', 'Tree', 'Vegetation', 'Building', 'Fence', 'Traffic Signals',
                  'Fire hydrant', 'Chair', 'Trash on roads', 'Trash bins', 'Person', 'Car']

In [262]:
def get_mAP(targets, preds):
    APs = np.zeros(preds.shape[1])
    ARs = np.zeros(preds.shape[1])
    for k in range(preds.shape[1]):
        APs[k], ARs[k] = _average_precision_2(targets[:, k], preds[:, k])

    return APs, ARs


def _average_precision_2(target, output):
    epsilon = 1e-8

    total_pred_pos = np.count_nonzero(output)
    total_target_pos = np.count_nonzero(target)

    TP = np.count_nonzero(np.logical_and(output, target))
    FP = total_pred_pos - TP
    FN = total_target_pos - TP

    AP = TP / (TP + FP + epsilon)
    AR = TP / (TP + FN + epsilon)

    return AP, AR


def calculate_model_map(gt_path, pred_path, obj_list=None):
    gt_dict = {}
    pred_dict = {}
    
    gt_files = [x for x in os.listdir(gt_path) if x.endswith('.csv') and int(x.split('-')[1])<=16]

    for fl in gt_files:
        if fl in ["video-10-segment-1.csv", "video-9-segment-2.csv"]:
            continue
        gt_fl = os.path.join(gt_path, fl)
        pred_fl = os.path.join(pred_path, fl)
        if not os.path.exists(pred_fl):
            continue

        gt_data = pd.read_csv(gt_fl)
        gt_data.replace(-1, 1, inplace=True) 
        x = []
        for index, row in gt_data.iterrows():
            if list(row)[0].strip() in ["Sidewalk pits"]:
                continue
            if list(row)[0].strip() in gt_dict.keys():
                gt_dict[list(row)[0].strip()] += list(row)[1:]
            else:
                gt_dict[list(row)[0].strip()] = list(row)[1:]

            x.append(len(list(row)[1:]))

        y = []    
        pred_data = pd.read_csv(pred_fl)
        pred_data.replace(-1, 1, inplace=True) 
        for index, row in pred_data.iterrows():
            if list(row)[0].strip() in ["Sidewalk pits"]:
                continue
            if list(row)[0].strip() in pred_dict.keys():
                pred_dict[list(row)[0].strip()] += list(row)[1:]
            else:
                pred_dict[list(row)[0].strip()] = list(row)[1:]
            y.append(len(list(row)[1:]))
            
        if x != y:
            print(len(x), len(y), fl)
    
    target_array = []
    pred_array = []

    for key in gt_dict.keys():
        if key in pred_dict.keys():
            if obj_list:
                if key not in obj_list:
                    continue
            if len(gt_dict[key]) == len(pred_dict[key]):
                target_array.append(gt_dict[key])
                pred_array.append(pred_dict[key])
            else:
                print(len(gt_dict[key]), len(pred_dict[key]), key)

    target_array = np.array(target_array).T
    pred_array = np.array(pred_array).T
    
    try:
        # ap = np.array(metrics.precision_score(target_array, pred_array, average=None))
        ap, ar = get_mAP(target_array, pred_array)
    except Exception as e:
        # ap = 0
        print(e)
        ap, ar = [0], [0]
        print(target_array.shape, pred_array.shape)
    
    return ap, ar

In [275]:
pred_fol = os.path.join(data_path, "BLIP")
apg, apg_2 = calculate_model_map(gt_fol, pred_fol)

print(apg.mean(), apg_2.mean())

0.23132030614136453 0.5664054341615655


In [276]:
pred_fol = os.path.join(data_path, "GPV-1")
apg, apg_2 = calculate_model_map(gt_fol, pred_fol)

print(apg.mean(), apg_2.mean())

0.19984905268298686 0.596441629626363


In [277]:
pred_fol = os.path.join(data_path, "yolo_v7")
apg, apg_2 = calculate_model_map(gt_fol, pred_fol)

print(apg.mean(), apg_2.mean())

0.1019004699310333 0.07751831077649515


In [278]:
pred_fol = os.path.join(data_path, "HRNet_V2")
apg, apg_2 = calculate_model_map(gt_fol, pred_fol)

print(apg.mean(), apg_2.mean())

0.08170082537543284 0.15335462191400137


In [279]:
pred_fol = os.path.join(data_path, "mask_rcnn")
apg, apg_2 = calculate_model_map(gt_fol, pred_fol)

print(apg.mean(), apg_2.mean())

0.056110478031014654 0.10562244944981927


In [280]:
pred_fol = os.path.join(data_path, "faster_rcnn")
apg, apg_2 = calculate_model_map(gt_fol, pred_fol)

print(apg.mean(), apg_2.mean())

0.05574902585860564 0.10317378290698756


In [281]:
pred_fol = os.path.join(data_path, "Random")
apg, apg_2 = calculate_model_map(gt_fol, pred_fol)

print(apg.mean(), apg_2.mean())

0.13666928138079906 0.3948667904701338
